In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [11]:
#Variable 
var="tg_mean"

files85 = glob.glob(folder+"*"+"rcp85_"+var+"_annual.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+var+"_annual.nc")
filesobs = glob.glob(folder+"NRCAN_obs_"+var+"_annual.nc")

#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45


['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_tg_mean_annual.nc']

In [12]:
filesobs

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NRCAN_obs_tg_mean_annual.nc']

# FOR RCP45 

In [51]:
dsEns45= ens.create_ensemble(files45)
#dsEns45

## Get the percentiles

In [7]:
# Calculate ensembles percentiles on regional means
perctiles45 = ens.ensemble_percentiles(dsEns45)
perctiles45
dsEns45 =0

## Convert in Dataframe

In [8]:
dfper145 = perctiles45.to_dataframe().dropna()
dfper45 = perctiles45.drop('realization').to_dataframe().dropna()
dfmean45 = dfper45.reset_index()
dfper45 = 0

In [50]:
#dfmean45

In [13]:
dfmean45.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_forTS.feather")

In [ ]:
dfmean45 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_forTS.feather")

# FOR RCP85 

In [52]:
dsEns85= ens.create_ensemble(files85)
#dsEns85

## Get the percentiles

In [15]:
# Calculate ensembles percentiles on regional means
perctiles85 = ens.ensemble_percentiles(dsEns85)
perctiles85
dsEns85 =0

## Convert in Dataframe

In [16]:
dfper185 = perctiles85.to_dataframe().dropna()
dfper85 = perctiles85.drop('realization').to_dataframe().dropna()
dfmean85 = dfper85.reset_index()
dfper85 = 0

In [53]:
#dfmean85

In [18]:
dfmean85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_forTS.feather")

In [ ]:
dfmean85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_forTS.feather")

# OBSERVED DATA

In [19]:
dsOBs= xr.open_mfdataset(filesobs)

will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  dsOBs= xr.open_mfdataset(filesobs)

to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  combined = auto_combine(



In [54]:
#dsOBs

## Convert in Dataframe

In [80]:
dfperObs = dsOBs.to_dataframe().dropna()
dfmeanObs = dfperObs.reset_index()
dfperObs = 0

In [82]:
dfObs = dfmeanObs.rename(columns={'tg_mean': 'tg_mean_Obs'})

In [83]:
dfObs

,lat,lon,time,tg_mean_Obs
0,83.125000,-74.541672,1950-01-01,251.429901
1,83.125000,-74.541672,1951-01-01,252.994797
2,83.125000,-74.541672,1952-01-01,253.964935
3,83.125000,-74.541672,1953-01-01,253.616013
4,83.125000,-74.541672,1954-01-01,253.628616
...,...,...,...,...
16464827,41.708332,-82.208336,2009-01-01,282.584686
16464828,41.708332,-82.208336,2010-01-01,283.870300
16464829,41.708332,-82.208336,2011-01-01,284.144257
16464830,41.708332,-82.208336,2012-01-01,285.570129


In [84]:
dfObs.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

In [ ]:
dfmeanObs = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

## Merge 2 rcp dataframes

In [44]:
df2m =  pd.merge(dfmean45, dfmean85, on=["lat","lon", "time"], suffixes=('_45', '_85'))

In [45]:
df2m

,lat,lon,time,tg_mean_p10_45,tg_mean_p50_45,tg_mean_p90_45,tg_mean_p10_85,tg_mean_p50_85,tg_mean_p90_85
0,66.623306,-89.045212,1950-01-01,257.085693,258.653290,260.071655,257.056732,258.643677,260.095032
1,66.623306,-89.045212,1951-01-01,256.835785,258.389740,259.078644,256.838013,258.383789,259.087128
2,66.623306,-89.045212,1952-01-01,257.146881,258.677917,259.168579,257.158539,258.634918,259.196228
3,66.623306,-89.045212,1953-01-01,257.813049,258.613068,259.164734,257.833679,258.571075,259.167938
4,66.623306,-89.045212,1954-01-01,258.522583,259.148529,259.997162,258.513580,259.165405,260.001831
...,...,...,...,...,...,...,...,...,...
11507101,40.041039,-74.045807,2096-01-01,287.267639,287.622437,289.489594,289.641998,290.565857,292.025970
11507102,40.041039,-74.045807,2097-01-01,286.741150,287.997528,288.674133,289.080017,290.811249,291.779266
11507103,40.041039,-74.045807,2098-01-01,286.326630,287.628967,288.842590,288.290466,290.388062,291.735809
11507104,40.041039,-74.045807,2099-01-01,286.937592,287.839813,290.203461,289.418976,290.153625,291.091431


# Cut by region for rcp45, 85 and Obs

## Get the lat lon regions

In [33]:
regions_folder = "/home/mlopez/EXEC/Grids-polygons-regions/"

In [49]:
files_regions = glob.glob(regions_folder+"*"+"lat_lon.feather")
files_regions

['/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_REG_FOR_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_UA_PER_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/REG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SDOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/Secteurs_Operations_Regionaleslat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SREG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/territoire_guidelat_lon.feather']

In [40]:
listdfreg=[]
for region in files_regions:
    df = pd.read_feather(region)
    listdfreg.append(df)
   

## Create DF by region

In [57]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [58]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [59]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

## Cutting rcp45 and 85 by region and getting df with 4th degree polynomial

In [67]:
def polysmooth(years, y, degree):
    coef = np.polyfit(years, y, degree)
    return np.polyval(coef,years)

In [115]:
listdf2m = []
for region, (name, short) in short_dict.items():
    print("------------------------------------------------------------------")
    print ("1/6... Cutting by region")
    print(region, name)
    dfregion = pd.read_feather("/home/mlopez/EXEC/Grids-polygons-regions/"+region+"lat_lon.feather")
    print ("2/6... Merging region with data")
    dfdatareg45 = pd.merge(dfregion, df2m, on=["lat","lon"])
    dfdataregObs = pd.merge(dfregion, dfObs, on=["lat","lon"])
    print ("3/6... Get mean by year for every sub-region")
    dfmeanUA45 = dfdatareg45
    dfdatareg45=0
    dfmean45UA = dfmeanUA45.groupby(["time", name]).mean()
    dfmean45UA2 = dfmean45UA.reset_index()
    dfmeanUA45=0
    dfmeanObs = dfdataregObs.groupby(["time", name]).mean() 
    dfdataregObs=0
    dfmeanObs2 = dfmeanObs.reset_index()
    dfmeanObs=0
    print ("4/6... Split by subregion")
    dfmean45UA3 = dfmean45UA2[['time', name, 'tg_mean_p10_45', 'tg_mean_p50_45', 'tg_mean_p90_45',
                               'tg_mean_p10_85', 'tg_mean_p50_85', 'tg_mean_p90_85']]
    dfmean45UA2= 0
    dfmeanObs3 = dfmeanObs2[['time', name, 'tg_mean_Obs']]
    dfmeanObs2 = 0
    for name2,sr in dfmeanObs3.groupby(name): 
        print ("                   "+name2, name)
        dfObssr = dfmeanObs3.loc[dfmeanObs3[name] == name2]
        dfObs1 = dfObssr.copy() 
        yearsO = dfObs1.time.dt.year
       # print (yearsO)
        dfp4O = pd.DataFrame()
        dfp4O["time"] = yearsO.values
        dfp4O[name] = name2
        dfp4O[var+"_Obs"] = dfObs1[var+"_Obs"] - 273.15 
        dfp42O = dfp4O.copy()
        dfp42O[var+"_Obs"] = round(dfp42[var+"_Obs"], 2)
    for name2,sr in dfmean45UA3.groupby(name):
        print ("                   "+name2, name)
        dfUAsr = dfmean45UA3.loc[dfmean45UA3[name] == name2]
        dfUA451 = dfUAsr.copy()
        print ("6/8... 4th Polynomial - rcp45,85")
        years = dfUA451.time.dt.year
        dfp4 = pd.DataFrame()
        dfp4["time"] = years.values
        dfp4[name] = name2
        for p in ('_p10_45', '_p50_45', '_p90_45', '_p10_85', '_p50_85', '_p90_85'):
            dfp4[var+p] = polysmooth(years,dfUA451[var+p],4) - 273.15
            dfp4[var+p] = round(dfp4[var+p], 2)
        #print (list(dfp4.columns.values))
    print("7/8 Merging rcp45,85 and observed")
    df3m = pd.merge(dfp4, dfp42O, how='left', on=["time", name])
    print (list(df3m.columns.values))
    print(" -----------------------8/8 Creating CSV file for sub-region " +name2)
    df3m.to_csv(replace_all(name2,d)+var+".csv")
                            
       

------------------------------------------------------------------
1/6... Cutting by region
DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR
2/6... Merging region with data
3/6... Get mean by year for every sub-region
4/6... Split by subregion
                   ABITIBI-TEMISCAMINGUE NM_REG_FOR
                   BAS-SAINT-LAURENT NM_REG_FOR
                   CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES NM_REG_FOR
                   COTE-NORD NM_REG_FOR
                   ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL NM_REG_FOR
                   GASPESIE-ILES-DE-LA-MADELEINE NM_REG_FOR
                   LANAUDIERE NM_REG_FOR
                   LAURENTIDES NM_REG_FOR
                   MAURICIE-CENTRE-DU-QUÉBEC NM_REG_FOR
                   NORD-DU-QUEBEC NM_REG_FOR
                   OUTAOUAIS NM_REG_FOR
                   SAGUENAY -LAC-SAINT-JEAN NM_REG_FOR
                   ABITIBI-TEMISCAMINGUE NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
                   BAS-SAINT-LAURENT NM_REG_FOR
6/8... 4th Polynomial -

2/6... Merging region with data


KeyboardInterrupt: 

In [99]:
listdf2m[3]

,time,NM_REG_FOR,tg_mean_p10_45,tg_mean_p50_45,tg_mean_p90_45,tg_mean_p10_85,tg_mean_p50_85,tg_mean_p90_85
0,1950,COTE-NORD,-2.70,-1.53,-0.54,-2.66,-1.62,-0.68
1,1951,COTE-NORD,-2.67,-1.54,-0.56,-2.64,-1.62,-0.68
2,1952,COTE-NORD,-2.65,-1.55,-0.59,-2.62,-1.62,-0.69
3,1953,COTE-NORD,-2.62,-1.56,-0.61,-2.60,-1.61,-0.69
4,1954,COTE-NORD,-2.60,-1.57,-0.64,-2.58,-1.61,-0.69
...,...,...,...,...,...,...,...,...
146,2096,COTE-NORD,0.69,2.54,3.99,3.78,5.96,7.63
147,2097,COTE-NORD,0.65,2.56,4.02,3.84,6.05,7.73
148,2098,COTE-NORD,0.62,2.57,4.04,3.89,6.14,7.83
149,2099,COTE-NORD,0.58,2.59,4.07,3.95,6.23,7.94


## DF for Observed Data

In [93]:
listdfObs = []
for region, (name, short) in short_dict.items():
    print("------------------------------------------------------------------")
    print ("1/5... Cutting by region")
    print(region, name)
    dfregion = pd.read_feather("/home/mlopez/EXEC/Grids-polygons-regions/"+region+"lat_lon.feather")
    print ("2/5... Merging region with data")
    dfdataregObs = pd.merge(dfregion, dfObs, on=["lat","lon"]) ##############
    print ("3/5... Get mean by year for every sub-region")
    dfmeanUA45 = dfdatareg45
    dfmeanObs = dfdataregObs.groupby(["time", name]).mean() #########################333
    dfmeanObs2 = dfmeanObs.reset_index()#######################
    print ("4/5... Split by subregion")
    dfmeanObs3 = dfmeanObs2[['time', name, 'tg_mean_Obs']]#######
    for name2,sr in dfmeanObs3.groupby(name): ########################
        print ("                   "+name2, name)
        dfObssr = dfmeanObs3.loc[dfmeanObs3[name] == name2]#################33333
        dfObs1 = dfObssr.copy() ######################
        print ("5/5... 4th Polynomial")
        yearsO = dfObs1.time.dt.year##############
        dfp4O = pd.DataFrame()###########
        dfp4O["time"] = years.values##########################
        dfp4O[name] = name2######################
        dfp4O[var+"_Obs"] = polysmooth(years,dfObs1[var+"_Obs"],4) - 273.15 ##########################
        dfp42O = dfp4O.copy()#######################
        dfp42O[var+"_Obs"] = round(dfp42[var+"_Obs"], 2)###########################
        listdfObs.append(dfp42)

------------------------------------------------------------------
1/5... Cutting by region
DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR
2/5... Merging region with data
3/5... Get mean by year for every sub-region
4/5... Split by subregion
                   ABITIBI-TEMISCAMINGUE NM_REG_FOR
5/5... 4th Polynomial
                   BAS-SAINT-LAURENT NM_REG_FOR
5/5... 4th Polynomial
                   CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES NM_REG_FOR
5/5... 4th Polynomial
                   COTE-NORD NM_REG_FOR
5/5... 4th Polynomial
                   ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL NM_REG_FOR
5/5... 4th Polynomial
                   GASPESIE-ILES-DE-LA-MADELEINE NM_REG_FOR
5/5... 4th Polynomial
                   LANAUDIERE NM_REG_FOR
5/5... 4th Polynomial
                   LAURENTIDES NM_REG_FOR
5/5... 4th Polynomial
                   MAURICIE-CENTRE-DU-QUÉBEC NM_REG_FOR
5/5... 4th Polynomial
                   NORD-DU-QUEBEC NM_REG_FOR
5/5... 4th Polynomial
                   OUTAOUAIS NM

5/5... 4th Polynomial
                   Hautes collines du réservoir aux Outardes NOM
5/5... 4th Polynomial
                   Massif des monts Groulx NOM
5/5... 4th Polynomial
                   Massif du lac Jacques-Cartier NOM
5/5... 4th Polynomial
                   Massif du mont Valin NOM
5/5... 4th Polynomial
                   Massif gaspésien NOM
5/5... 4th Polynomial
                   Plaine de l'Abitibi NOM
5/5... 4th Polynomial
                   Plaine de la baie de Rupert NOM
5/5... 4th Polynomial
                   Plaine du Saint-Laurent NOM
5/5... 4th Polynomial
                   Plaine du bas Outaouais et de l'archipel de Montréal NOM
5/5... 4th Polynomial
                   Plaine du lac Matagami NOM
5/5... 4th Polynomial
                   Plaine du lac Opémisca NOM
5/5... 4th Polynomial
                   Plaine du lac Saint-Jean et du Saguenay NOM
5/5... 4th Polynomial
                   Plaines et coteaux du lac Simard NOM
5/5... 4th Polynomial
               

3/5... Get mean by year for every sub-region
4/5... Split by subregion
                   1a TER_GUIDE
5/5... 4th Polynomial
                   2a TER_GUIDE
5/5... 4th Polynomial
                   2b TER_GUIDE
5/5... 4th Polynomial
                   2c TER_GUIDE
5/5... 4th Polynomial
                   3ab TER_GUIDE
5/5... 4th Polynomial
                   3c TER_GUIDE
5/5... 4th Polynomial
                   3d TER_GUIDE
5/5... 4th Polynomial
                   4a TER_GUIDE
5/5... 4th Polynomial
                   4bc TER_GUIDE
5/5... 4th Polynomial
                   4de TER_GUIDE
5/5... 4th Polynomial
                   4f TER_GUIDE
5/5... 4th Polynomial
                   4gh TER_GUIDE
5/5... 4th Polynomial
                   5a TER_GUIDE
5/5... 4th Polynomial
                   5bcd TER_GUIDE
5/5... 4th Polynomial
                   5ef TER_GUIDE
5/5... 4th Polynomial
                   5g TER_GUIDE
5/5... 4th Polynomial
                   5hi TER_GUIDE
5/5... 4th Polynomial
   

In [95]:
listdfObs[1]

,time,NM_REG_FOR,tg_mean_Obs
0,1950,BAS-SAINT-LAURENT,2.76
1,1951,BAS-SAINT-LAURENT,2.69
2,1952,BAS-SAINT-LAURENT,2.62
3,1953,BAS-SAINT-LAURENT,2.57
4,1954,BAS-SAINT-LAURENT,2.51
...,...,...,...
59,2009,BAS-SAINT-LAURENT,3.44
60,2010,BAS-SAINT-LAURENT,3.52
61,2011,BAS-SAINT-LAURENT,3.60
62,2012,BAS-SAINT-LAURENT,3.68


In [ ]:
 
        TO-DO: 
            - Same for observed data?
            - merge all 3 df
            print to csv
        
    
    
    
    
    
    
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftgseall3, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_"+var+"_monthly.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_"+var+"_monthly.json", driver="GeoJSON")

In [ ]:
df45

### TODO:
- This dataframe should include both models (RCP 4.5 nad 8.5) and observations (historic)
- I should add labels and similar colors into the shiny graph

In [ ]:
df45.to_csv("p4tgmean.csv")

In [ ]:
plt.plot(years, df45["tg_mean_p50"], markersize='10', color='blue')
plt.fill_between(years, polysmooth(years, df45.tg_mean_p10,4), polysmooth(years, df45.tg_mean_p90,4), alpha=0.3, color='blue')

In [ ]:
dfmean2.to_csv("timeseries.csv")

### NOTE: this example was for the whole region. 
### TODO: 
- Split DF by regions and get same graph for each region
- Convert degrees in Celsius